In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found

In [2]:
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)

from peft import get_peft_model, LoraConfig
import pandas as pd
from huggingface_hub import login
from torch import float16

2025-05-24 17:10:16.166197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748106616.400638      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748106616.470239      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct" # for LLaMA 3.1
# MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"     # for Mistral

In [4]:
train_df = pd.read_csv("/kaggle/input/summarization/cnn_dailymail/train.csv", engine="python", on_bad_lines='skip', nrows=1000)
test_df = pd.read_csv("/kaggle/input/summarization/cnn_dailymail/test.csv", engine="python", on_bad_lines='skip')

In [5]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
login() #login using hf token with access to llama and mistral models

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  
    bnb_4bit_compute_dtype=float16, 
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [10]:
# Apply LoRA
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

In [11]:
def preprocess_function(example):
    instruction = "### Instruction:\nSummarize the following article."
    input_text = f"\n\n### Input:\n{example['article']}"
    response = f"\n\n### Response:\n{example['highlights']}{tokenizer.eos_token}"

    full_prompt = instruction + input_text + response

    tokenized = tokenizer(
        full_prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None
    )

    # Identify the start index of the response for label masking
    response_start = full_prompt.index("### Response:\n") + len("### Response:\n")
    response_token_start = len(tokenizer(full_prompt[:response_start], truncation=True, max_length=512)["input_ids"])

    # Initialize labels with -100
    labels = [-100] * len(tokenized["input_ids"])
    labels[response_token_start:] = tokenized["input_ids"][response_token_start:]
    tokenized["labels"] = labels

    return tokenized

In [12]:
train_tokenized = train_dataset.map(preprocess_function, remove_columns=train_dataset.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to='none',
    logging_steps=1,
    label_names=["labels"]
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
)

In [15]:
trainer.train()

Step,Training Loss
1,1.653100
2,1.087900
3,1.009700
4,0.760000
5,0.920000
6,0.465600
7,0.779100
8,0.538100
9,0.504800
10,0.588800


TrainOutput(global_step=62, training_loss=0.589133977409332, metrics={'train_runtime': 4746.8475, 'train_samples_per_second': 0.421, 'train_steps_per_second': 0.013, 'total_flos': 4.52701288660992e+16, 'train_loss': 0.589133977409332, 'epoch': 1.96})

In [16]:
model.save_pretrained("fine-tuned-model")
tokenizer.save_pretrained("fine-tuned-model")

('fine-tuned-model/tokenizer_config.json',
 'fine-tuned-model/special_tokens_map.json',
 'fine-tuned-model/tokenizer.json')

In [24]:
# Interactive loop
print("Enter an article or paragraph to summarize (type 'exit' to quit):")

user_input = input("\nArticle: ").strip()

prompt = f"### Instruction:\nSummarize the following article.\n\n### Input:\n{user_input}\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens = 512
    )

generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
summary = generated.split("### Response:")[-1].strip()

print(f"\nSummary:\n{summary}")

Enter an article or paragraph to summarize (type 'exit' to quit):



Article:  (CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as "The Milk Man," of using his role as a police officer to help the drug trafficking organization in exchange for money and gifts, including a Rolex watch. In one instance, the complaint alleges, Mata arranged to pay two assassins to kill rival drug dealers. The killers would pose as cops, pulling over their targets before shooting them, according to the complaint. "Ultimately, the (organization) decided not to move forward with the murder plot, but Mata still received a payment for setting up the meetings," federal prosecutors said in a statement. T

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Summary:
Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department.
He allegedly worked with a drug trafficking organization to help plan a murder plot.
Mata faces charges of aiding and abetting a conspiracy to distribute cocaine.


In [23]:
print(train_df['article'][1],end="\n\n")
print(train_df['highlights'][1])

(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as "The Milk Man," of using his role as a police officer to help the drug trafficking organization in exchange for money and gifts, including a Rolex watch. In one instance, the complaint alleges, Mata arranged to pay two assassins to kill rival drug dealers. The killers would pose as cops, pulling over their targets before shooting them, according to the complaint. "Ultimately, the (organization) decided not to move forward with the murder plot, but Mata still received a payment for setting up the meetings," federal prosecutors said in a statement. The complain